<a href="https://colab.research.google.com/github/naratiz/Precoce-MS/blob/main/4_Modeling_Classifica%C3%A7%C3%A3o/Otimiza%C3%A7%C3%A3o_Classifica%C3%A7%C3%A3o_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install optuna

In [1]:
import optuna
import pandas as pd
import numpy as np
import logging
import sys
import time
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.utils import class_weight
from sklearn.model_selection import cross_val_score,train_test_split, StratifiedKFold

from catboost import CatBoostClassifier

In [2]:
import os 
n_cpu = os.cpu_count()
print("Number of CPUs in the system:", n_cpu)

Number of CPUs in the system: 12


# Base de Dados

In [2]:
path = '../../dados/df_train_class.csv'
df = pd.read_csv(path, sep=';')

In [4]:
print('A base de dados apresenta {} registros e {} características.'.format(df.shape[0], df.shape[1]))

A base de dados apresenta 672657 registros e 48 características.


### Separando as características preditoras do atributo alvo

In [3]:
SEED = 42

In [4]:
y = df['categoria']
# Retirar a variável alvo e as variáveis obtidas após o abate 
X = df.drop(['categoria'],axis=1)

In [5]:
# Para remover UserWarning do XGBClassifier: Use o parêmetro use_label_encoder=False 
# e codifique as classes no inervalo[0 ... n]
y=y.replace(6, 0)

### StandardScaler

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## PCA - Não compilado

In [15]:
pca = PCA(n_components=30)
pca.fit(X_scaled)
X_scaled_pca = pca.transform(X_scaled)

## Baseline

In [7]:
class_weight = class_weight.compute_class_weight(class_weight = "balanced",
                                                 classes = np.unique(y),
                                                 y = y)

In [16]:
skfolds = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)

In [17]:
cbc_baseline = CatBoostClassifier(loss_function='MultiClass', class_weights= class_weight,
                                  iterations=10, depth=10, random_state=SEED, verbose=False)
score = cross_val_score(cbc_baseline, X_scaled, y, cv = skfolds)

print('Acurácia média da validação cruzada : %0.3f"' % score.mean())

Acurácia média da validação cruzada : 0.375"


## Otimização dos hiperparâmetros

In [8]:
skfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

In [9]:
# Manipulador de fluxo para mostrar as mensagens
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "cbc-study"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
# storage_name = "sqlite:///content/drive/MyDrive/Dissertação/dados/{}.db".format(study_name)

In [10]:
# função de otimização baseada em uma função objetivo
def tune (objective):
    study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize")
    study.optimize(objective, timeout=86400, n_trials=300)

    params = study.best_params
    print(f"Trial: {study.best_trial.number}")
    params = study.best_params
    print("\n")
    print(f"Melhor seleção de parâmetros : \n{params}")
    print("\n")
    print(f"Acurácia : {study.best_value:.3f}")
    
    return study, params

# CatBoostClassifier

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=SEED)

In [12]:
def cbc_objective(trial: optuna.Trial): 
    cbc_params = {
        'loss_function':'MultiClass',
        'iterations': trial.suggest_int('iterations', 1, 12),
        'depth': trial.suggest_int('depth', 1, 12),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-5, 1e0),
        'l2_leaf_reg': trial.suggest_loguniform("l2_leaf_reg", 1e-2, 1e0),
        'random_state': SEED
    }
    model = CatBoostClassifier(**cbc_params, class_weights= class_weight, verbose=False) 
    scores = cross_val_score(model, X_train, y_train, n_jobs=-1, cv=skfolds)
    return scores.mean()

In [13]:
t0 = time.time()
study, best_params = tune(cbc_objective)  
print("Pronto em:  %0.3f" %(time.time()-t0))

[I 2022-07-13 23:33:43,150] Using an existing study with name 'cbc-study' instead of creating a new one.


Using an existing study with name 'cbc-study' instead of creating a new one.


[I 2022-07-13 23:33:56,094] Trial 1 finished with value: 0.23673965933869864 and parameters: {'iterations': 10, 'depth': 6, 'learning_rate': 1.4873900082708847e-05, 'l2_leaf_reg': 0.0316106290926731}. Best is trial 1 with value: 0.23673965933869864.


Trial 1 finished with value: 0.23673965933869864 and parameters: {'iterations': 10, 'depth': 6, 'learning_rate': 1.4873900082708847e-05, 'l2_leaf_reg': 0.0316106290926731}. Best is trial 1 with value: 0.23673965933869864.


[I 2022-07-13 23:34:04,169] Trial 2 finished with value: 0.22538806756583285 and parameters: {'iterations': 3, 'depth': 3, 'learning_rate': 1.557221451776181e-05, 'l2_leaf_reg': 0.23414166480444998}. Best is trial 1 with value: 0.23673965933869864.


...

Trial 298 finished with value: 0.41711212254900715 and parameters: {'iterations': 12, 'depth': 12, 'learning_rate': 0.8297182508335615, 'l2_leaf_reg': 0.3151830342775885}. Best is trial 226 with value: 0.4186561147842326.


[I 2022-07-14 05:31:40,737] Trial 299 finished with value: 0.4121913424135514 and parameters: {'iterations': 12, 'depth': 12, 'learning_rate': 0.9975310859291643, 'l2_leaf_reg': 0.26014994793396395}. Best is trial 226 with value: 0.4186561147842326.


Trial 299 finished with value: 0.4121913424135514 and parameters: {'iterations': 12, 'depth': 12, 'learning_rate': 0.9975310859291643, 'l2_leaf_reg': 0.26014994793396395}. Best is trial 226 with value: 0.4186561147842326.


[I 2022-07-14 05:31:51,830] Trial 300 finished with value: 0.24800418135872002 and parameters: {'iterations': 12, 'depth': 2, 'learning_rate': 0.4820705946554799, 'l2_leaf_reg': 0.3138199923968319}. Best is trial 226 with value: 0.4186561147842326.


Trial 300 finished with value: 0.24800418135872002 and parameters: {'iterations': 12, 'depth': 2, 'learning_rate': 0.4820705946554799, 'l2_leaf_reg': 0.3138199923968319}. Best is trial 226 with value: 0.4186561147842326.
Trial: 226


Melhor seleção de parâmetros : 
{'depth': 12, 'iterations': 12, 'l2_leaf_reg': 0.2982072500524196, 'learning_rate': 0.802149821813751}


Acurácia : 0.419
Pronto em:  21488.882


In [ ]:
# Determina quais parâmetros tiveram o impacto mais significativo no desempenho geral do modelo.
optuna.visualization.plot_param_importances(study)